![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import joblib

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

In [ ]:
# Visualización datos de test
dataTesting.head()

In [ ]:
# Valores negativos en variable de respuesta
dataTraining.Price[dataTraining.Price<=0].count()

In [ ]:
# Encoding variables categóricas
for c in dataTraining.columns:
    if dataTraining[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(dataTraining[c].values)) 
        dataTraining[c] = lbl.transform(list(dataTraining[c].values))

In [ ]:
# Encoding variables categóricas
for c in dataTesting.columns:
    if dataTesting[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(dataTesting[c].values)) 
        dataTesting[c] = lbl.transform(list(dataTesting[c].values))
        
dataTesting.head()

In [ ]:
#Carga de datos
XTotal = dataTraining.drop(['Price'], axis=1)#.to_numpy()
yTotal = dataTraining['Price']#.to_numpy()

#Partición de datos
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.33, random_state=2)

In [ ]:
# Entrenamiento de modelo
regressor = XGBRegressor()
regressor.fit(XTrain, yTrain)
#y_pred = regressor.predict(XTest)
#rms = mean_squared_error(yTest, y_pred, squared=False)
#rms

In [ ]:
# Exportar modelo a archivo binario .pkl
joblib.dump(regressor, './model_deployment/car_price.pkl', compress=3)

In [ ]:
# Importar modelo y predicción
from model_deployment.xx_model_deployment import pred_car_price

# Predicción de probabilidad
pred_car_price(2014,31909,20,26,305) #18501.033

In [ ]:
#!/usr/bin/python
from flask import Flask
from flask_restx import Api, Resource, fields
import joblib
#from xx_model_deployment import pred_car_price
from model_deployment.xx_model_deployment import pred_car_price

app = Flask(__name__)

api = Api(
    app,
    version='1.0',
    title='Car Price Prediction API',
    description='Car Price Prediction API')

ns = api.namespace('predict',
     description='Car Price Regressor')

parser = api.parser()

parser.add_argument(
    'Year',
    type=float,
    required=True,
    help='Year of the car',
    location='args')

parser.add_argument(
    'Mileage',
    type=float,
    required=True,
    help='Mileage of the car',
    location='args')

parser.add_argument(
    'State',
    type=float,
    required=True,
    help='State of the car',
    location='args')

parser.add_argument(
    'Make',
    type=float,
    required=True,
    help='Make of the car',
    location='args')

parser.add_argument(
    'Model',
    type=float,
    required=True,
    help='Model of the car',
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.Float,
})

@ns.route('/')
class CarPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()

        return {
         "result": pred_car_price(args['Year'],args['Mileage'],args['State'],args['Make'],args['Model'])
        }, 200

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8888)